# Notebook for data gathering
## Setting up dependencies, keys and such

In [1]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint

# Attom API key
from config import attom_data


## Testing out the APIs, looking and results, seeing what we want/need

In [2]:
# Testing out an API - for school district info
base_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/school/districtdetail?id="
school_id = "5700086424"
url = base_url + school_id
r=requests.get(url, headers={"accept":"application/json","apikey":attom_data})
r

<Response [200]>

In [3]:
# Looking at the JSON data for school data we get back so we can decide what we need
school_data = r.json()
school_data

{'status': {'version': '1.0.0',
  'code': 0,
  'msg': 'SuccessWithResult',
  'total': 1,
  'page': 1,
  'pagesize': 10,
  'responseDateTime': None,
  'transactionID': None},
 'school': [{'Identifier': {'Obdistrictnumber': '5700086424'},
   'ProfilesAndTestScores': {'DistrictSummary': {'districttype': 'DISTRICT IN SUPERVISORY UNION',
     'districtname': 'LYME SCHOOL DISTRICT',
     'county3': '009',
     'countyName': 'GRAFTON COUNTY',
     'county': '33009',
     'latitude': 43.811031,
     'longitude': -72.159683,
     'locationaddress': 'RTE 10',
     'locationcity': 'LYME',
     'fipsState': '33',
     'stateabbrev': 'NH',
     'zip54': '03768',
     'zip': '03768',
     'phone': '603-795-2125',
     'startDate': '08/29',
     'endDate': '06/15',
     'GStestrating': 5},
    'DistrictContact': {'Prefixliteral': 'Mr',
     'Firstname': 'Jeff',
     'Lastname': 'Valence',
     'Gender': 'M',
     'Englishtitle': 'Superintendent',
     'Websiteurl': 'http://www.lymeschool.org'},
    '

In [4]:
# Testing out an API for sales info
#"name": "Sales (snapshot) by postal code",

base_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/sale/snapshot?postalcode=63123&page=1&pageSize=20"
s = requests.get(base_url, headers={"accept":"application/json", "apikey":attom_data})
s

<Response [200]>

In [5]:
#Looking at the JSON data for sales data
sales_data = r.json()
sales_data

{'status': {'version': '1.0.0',
  'code': 0,
  'msg': 'SuccessWithResult',
  'total': 1,
  'page': 1,
  'pagesize': 10,
  'responseDateTime': None,
  'transactionID': None},
 'school': [{'Identifier': {'Obdistrictnumber': '5700086424'},
   'ProfilesAndTestScores': {'DistrictSummary': {'districttype': 'DISTRICT IN SUPERVISORY UNION',
     'districtname': 'LYME SCHOOL DISTRICT',
     'county3': '009',
     'countyName': 'GRAFTON COUNTY',
     'county': '33009',
     'latitude': 43.811031,
     'longitude': -72.159683,
     'locationaddress': 'RTE 10',
     'locationcity': 'LYME',
     'fipsState': '33',
     'stateabbrev': 'NH',
     'zip54': '03768',
     'zip': '03768',
     'phone': '603-795-2125',
     'startDate': '08/29',
     'endDate': '06/15',
     'GStestrating': 5},
    'DistrictContact': {'Prefixliteral': 'Mr',
     'Firstname': 'Jeff',
     'Lastname': 'Valence',
     'Gender': 'M',
     'Englishtitle': 'Superintendent',
     'Websiteurl': 'http://www.lymeschool.org'},
    '

In [6]:
# Another API - for school detail
# School detail - curl -X GET --header 'Accept: application/json' --header 'apikey: 736f1130096aa92549d800921bca8e8c' 'https://api.gateway.attomdata.com/propertyapi/v1.0.0/school/detail?id=00582204'
sd_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/school/detail?id=00582204"

#sd = requests.get(base_url, headers={"accept":"application/json", "apikey":attom_data})
sd = requests.get(sd_url, headers={"Accept":"application/json", "apikey":attom_data})
sd

<Response [200]>

In [7]:
# Looking at the JSON data for school detail
school_detail = sd.json()
school_detail

{'status': {'version': '1.0.0',
  'code': 0,
  'msg': 'SuccessWithResult',
  'total': 1,
  'page': 1,
  'pagesize': 10,
  'responseDateTime': None,
  'transactionID': None},
 'school': [{'Identifier': {'OBInstID': '00582204'},
   'SchoolProfileAndDistrictInfo': {'SchoolLocation': {'COUNTY3': '510',
     'COUNTYNAME': 'ST. LOUIS CITY',
     'COUNTY': '29510',
     'geocodinglatitude': 38.610114,
     'geocodinglongitude': -90.218684,
     'locationaddress': '2156 RUSSELL BLVD',
     'locationcity': 'SAINT LOUIS',
     'fipsState': '29',
     'stateabbrev': 'MO',
     'zip54': '63104-2607',
     'ZIP': '63104',
     'Obdistrictnumber': '2900580749',
     'districttype': 'REGULAR LOCAL SCHOOL DISTRICT',
     'districtname': 'ST. LOUIS PUBLIC SCHOOLS',
     'geoid': None},
    'SchoolSummary': {'Filetypetext': 'PUBLIC',
     'buildingtypetext': 'REGULAR SCHOOL',
     'institutionname': 'MCKINLEY CLASS. LEADERSHIP AC.',
     'startDate': '08/13',
     'endDate': '05/21',
     'gradelevel1lo


## Runing the APIs to get our data
####  Note: we found from the doc and experimenting that we'd need an OBInstID for each school to get detail info.  
####             We needed lat & lng info to get OBInstID -  see School_zips Jupyter notebook for zips to lat, lng using Google API
####             We gathered those OBInstIDs doing manual lookups on the ATTOM API website (there were multiple schools per zip)
####             Similar process for Real Estate sales info - there were numerous GeoIDs per zip, until we realized ZI##### worked
####             A "final" spreadsheet with zip, lat, lng, OBInstID and GeoID was created to use as a source for our ATTOM API calls

### Start with the School detail

In [8]:
# Import the csv file with OBInstID s in it to run with the school detail API
latlon_df = pd.read_csv("../Data/school_zip_lat_long2.csv", usecols=[0,1,3,4,5,6,7])
print(latlon_df.dtypes)
latlon_df.head()

Zip Codes            int64
Areas               object
Latitude           float64
Longitude          float64
School District     object
OBInstlD             int64
GeoID               object
dtype: object


,Zip Codes,Areas,Latitude,Longitude,School District,OBInstlD,GeoID
0,63123,Affton,38.558084,-90.327783,AFFTON HIGH SCHOOL,577405,ZI63123
1,63011,Ballwin,38.604451,-90.558879,WEST HIGH SCHOOL,579439,ZI63011
2,63021,Ballwin,38.557323,-90.535236,PARKWAY SOUTH HIGH SCHOOL,1524889,ZI63021
3,63137,Bellefontaine Neighbors,38.747943,-90.211357,RIVERVIEW GARDENS SR. HIGH SCHOOL,579881,ZI63137
4,63118,Benton Park,38.595062,-90.229156,ROOSEVELT HIGH SCHOOL,582008,ZI63118


In [9]:
# Set a variable for the school detail we want
school_details = []


# from the import of the csv file (done in cell above) - need OBInstID in either a list or df
# **NOTE using list temporarily to test
# OBInstID_list = ['577405','1524889']
OBInstID_list = latlon_df['OBInstlD'].tolist()

# Set School detail url
# sd_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/school/detail?id=00582204"
sd_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/school/detail?id="

# Loop over the list of IDs for each school and grab the detail information
for OBID in OBInstID_list:
    sd = requests.get(sd_url + str(OBID), headers={"accept":"application/json", "apikey":attom_data}).json()
    school = sd['school'][0]
    school_detail = school['SchoolProfileAndDistrictInfo']['SchoolDetail']
    school_details.append(school_detail)
    #print(sd)
    #print(base_url + OBID)
    

# Create a dataframe from the list of school detail info
school_details_df = pd.DataFrame(school_details)

# Add the school ID as a column to the df in case needed it later as a key for a merge
school_details_df["OBInstID"] = OBInstID_list

In [10]:
#print(OBInstID_list)
school_details_df.head()

,educationClimateIndex,advancedPlacement,beforeandafterschoolprgms,blueribbonschool,charterschools,giftedandtalented,internationbaccalaureate,magnetschool,sitebased,collegebound,...,adulteducationclasses,adultother,specialeducation,alterantiveprogram,yearroundclasses,ESL,Povertylevel,Instructionalexpensepupil,AYPschool,OBInstID
0,BELOW AVERAGE,Y,N,N,N,N,N,N,N,91,...,N,None,Y,N,N,Y,6 - 15.9 PERCENT,5669,N,577405
1,ABOVE AVERAGE,Y,Y,N,N,Y,N,N,N,98,...,N,None,Y,N,N,N,6 - 15.9 PERCENT,6840,N,579439
2,ABOVE AVERAGE,Y,N,N,N,Y,N,N,Y,95,...,Y,Y,Y,N,N,Y,6 - 15.9 PERCENT,6840,Y,1524889
3,BELOW AVERAGE,N,N,N,N,N,N,N,N,77,...,N,None,Y,N,N,N,6 - 15.9 PERCENT,4795,N,579881
4,LOW,Y,N,N,N,N,N,N,N,52,...,N,None,Y,N,N,Y,30 PERCENT OR MORE,7305,N,582008


In [11]:
# Write the school details data to a csv
school_details_df.to_csv("..\Data\school_detail2.csv", encoding="utf-8", index=False)


### Get the Real Estate sales data

In [12]:
# Grab the GeoID from the CSV we imported earlier (same CSV that also had school ID)
GeoID_list = latlon_df['GeoID'].tolist()
print(GeoID_list)

['ZI63123', 'ZI63011', 'ZI63021', 'ZI63137', 'ZI63118', 'ZI63134', 'ZI63144', 'ZI63044', 'ZI63005', 'ZI63017', 'ZI63105', 'ZI63128', 'ZI63126', 'ZI63141', 'ZI63131', 'ZI63045', 'ZI63025', 'ZI63026', 'ZI63135', 'ZI63031', 'ZI63033', 'ZI63042', 'ZI63136', 'ZI63140', 'ZI63122', 'ZI63124', 'ZI63125', 'ZI63143', 'ZI63043', 'ZI63121', 'ZI63129', 'ZI63034', 'ZI63132', 'ZI63114', 'ZI63133', 'ZI63115', 'ZI63117', 'ZI63101', 'ZI63102', 'ZI63103', 'ZI63104', 'ZI63106', 'ZI63107', 'ZI63108', 'ZI63109', 'ZI63110', 'ZI63111', 'ZI63112', 'ZI63113', 'ZI63116', 'ZI63139', 'ZI63146', 'ZI63147', 'ZI63155', 'ZI63138', 'ZI63074', 'ZI63127', 'ZI63130', 'ZI63088', 'ZI63120', 'ZI63119', 'ZI63038', 'ZI63040']


In [14]:
# Getting the real estate data
# curl -X GET --header 'Accept: application/json' --header 'apikey: 736f1130096aa92549d800921bca8e8c' 'https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?geoid=ZI63021&interval=yearly&startyear=2014&endyear=2018'

# import the csv file - need GeoID in either a list or df
# **NOTE using manual list temporarily to test
#GeoID_list = ['ZI63045']
GeoID_list = latlon_df['GeoID'].tolist()

# Set variables
sales_details2018 = []

# Set Real estate detail url
# re_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?geoid=ZI63021&interval=yearly&startyear=2014&endyear=2018"
re_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?interval=yearly&startyear=2014&endyear=2018&geoid="

# Loop over the list of GeoIDs representing each area and grab the sales trend information
try:
    for GeoID in GeoID_list:
        #re = requests.get(re_url + GeoID, headers={"accept":"application/json", "apikey":attom_data}).json()
        re = requests.get(re_url + str(GeoID), headers={"accept":"application/json", "apikey":attom_data}).json()
        sales2018 = re['salestrends'][4]
        sales_detail2018 = sales2018['SalesTrend']
        sales_details2018.append(sales_detail2018)
        #pprint(re)
        #print(base_url + GeoID)
# print(sales2014)
# print(sales2015)
except IndexError:
    print("IndexError on record: " + GeoID)
    sales_details2018.append({'homesalecount':'issue here', 'avgsaleprice':'issue here', 'medsaleprice':'issue here'})
    pass

# Create a df for the sales trend data
sales_details2018_df = pd.DataFrame(sales_details2018)

# Add the GeoID as a column to the df in case needed it later as a key for a merge
# sales_details2018_df["GeoID"] = GeoID_list

sales_details2018_df

IndexError on record: ZI63045


,homesalecount,avgsaleprice,medsaleprice
0,1128,157649,145950
1,696,294492,280000
2,1025,251951,227500
3,356,67045,41725
4,605,146375,114000
5,177,58138,50000
6,319,238304,185000
7,174,163868,165000
8,342,614509,566750
9,791,370493,349750


####  Errored out, so implemented error checking with try: except:
####  Printed the error (see above) & investigated to see what was going on.  
####  Turns out that zip 63045 had no salestrend data - i.e. the call was successful, just no results.

In [15]:
GeoID_list = ['ZI63045']
re_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?interval=yearly&startyear=2018&endyear=2018&geoid="

# Loop over the list of GeoIDs representing each area and grab the sales trend information - for the past 5 years
for GeoID in GeoID_list:
    #re = requests.get(re_url + GeoID, headers={"accept":"application/json", "apikey":attom_data}).json()
    re = requests.get(re_url + str(GeoID), headers={"accept":"application/json", "apikey":attom_data}).json()
    
pprint(re)

{'salestrends': [],
 'status': {'code': 1,
            'msg': 'SuccessWithoutResult',
            'total': 0,
            'version': '1.0.0'}}


In [18]:
# No salestrend data for ZI63045 - so remove it from our list . . .
# Same thing happened with ZI63140, and ZI63155.  remove them too.
GeoID_list = latlon_df['GeoID'].tolist()
GeoID_list.remove('ZI63045')
GeoID_list.remove('ZI63140')
GeoID_list.remove('ZI63155')
#print(GeoID_list)

# Then try the process again:
# Reset output list
sales_details2018 = []

# Set Real estate detail url
# re_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?geoid=ZI63021&interval=yearly&startyear=2014&endyear=2018"
re_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/salestrend/snapshot?interval=yearly&startyear=2014&endyear=2018&geoid="

# Loop over the list of GeoIDs representing each area and grab the sales trend information
#   Note to self - original plan 4 yrs of sales, but just have 1 for school, so go w/ 2018, i.e. index 4 (5th yr, 0 base)
try:
    for GeoID in GeoID_list:
        #re = requests.get(re_url + GeoID, headers={"accept":"application/json", "apikey":attom_data}).json()
        re = requests.get(re_url + str(GeoID), headers={"accept":"application/json", "apikey":attom_data}).json()
        sales2018 = re['salestrends'][4]
        sales_detail2018 = sales2018['SalesTrend']
        sales_details2018.append(sales_detail2018)
        
except IndexError:
    print("IndexError on record: " + GeoID)
    sales_details2018.append({'homesalecount':'issue here', 'avgsaleprice':'issue here', 'medsaleprice':'issue here'})
    pass

# Create a df for the sales trend data
sales_details2018_df = pd.DataFrame(sales_details2018)

# Add the GeoID as a column to the df in case needed it later as a key for a merge
sales_details2018_df["GeoID"] = GeoID_list

sales_details2018_df.head()

,homesalecount,avgsaleprice,medsaleprice,GeoID
0,1128,157649.0,145950.0,ZI63123
1,696,294492.0,280000.0,ZI63011
2,1025,251951.0,227500.0,ZI63021
3,356,67045.0,41725.0,ZI63137
4,605,146375.0,114000.0,ZI63118


In [19]:
# Write the sales trend data to csv
sales_details2018_df.to_csv("..\Data\sales2_details2018.csv", encoding="utf-8", index=False)

## Cleanup
#### Since we had to drop some zip codes due to no salestrend data, we also need to drop them from our school info.  Otherwise our samples will have different lengths and we won't be able to plot them against each other

In [ ]:
# Need to remove 63045, 63140, and 63155 from school_details_df


In [22]:
latlon_df.head()

,Zip Codes,Areas,Latitude,Longitude,School District,OBInstlD,GeoID
0,63123,Affton,38.558084,-90.327783,AFFTON HIGH SCHOOL,577405,ZI63123
1,63011,Ballwin,38.604451,-90.558879,WEST HIGH SCHOOL,579439,ZI63011
2,63021,Ballwin,38.557323,-90.535236,PARKWAY SOUTH HIGH SCHOOL,1524889,ZI63021
3,63137,Bellefontaine Neighbors,38.747943,-90.211357,RIVERVIEW GARDENS SR. HIGH SCHOOL,579881,ZI63137
4,63118,Benton Park,38.595062,-90.229156,ROOSEVELT HIGH SCHOOL,582008,ZI63118


In [63]:
# Pull the zip codes from the latlon df . . .
zip_list = latlon_df['Zip Codes'].tolist()

# Then add them (the zip codes) as a column to the School df so we can remove the 3 entries to match the real estate df
school_details_df["Zip_Codes"] = zip_list
school_details_df.tail()

,educationClimateIndex,advancedPlacement,beforeandafterschoolprgms,blueribbonschool,charterschools,giftedandtalented,internationbaccalaureate,magnetschool,sitebased,collegebound,...,adultother,specialeducation,alterantiveprogram,yearroundclasses,ESL,Povertylevel,Instructionalexpensepupil,AYPschool,OBInstID,Zip_Codes
58,AVERAGE,Y,Y,N,N,Y,N,N,N,94,...,Y,Y,N,N,Y,6 - 15.9 PERCENT,7839,Y,580359,63088
59,LOW,Y,N,N,N,N,N,N,N,84,...,None,Y,N,N,N,30 PERCENT OR MORE,7305,N,5700081307,63120
60,ABOVE AVERAGE,Y,Y,N,N,Y,N,N,Y,96,...,Y,Y,N,N,Y,6 - 15.9 PERCENT,7081,N,580517,63119
61,ABOVE AVERAGE,N,Y,N,N,Y,N,N,Y,0,...,Y,Y,N,N,Y,6 - 15.9 PERCENT,5763,N,4945866,63038
62,ABOVE AVERAGE,Y,N,N,N,Y,N,N,N,96,...,None,Y,N,N,Y,6 - 15.9 PERCENT,5763,N,580036,63040


In [48]:
#To select rows whose column value is in list 
zipstoremove = [63045, 63140, 63155]
schIDtoremove_list = []
removal_list_df = latlon_df.loc[latlon_df['Zip Codes'].isin(zipstoremove)]
schIDtoremove_list = removal_list_df['OBInstlD'].tolist()
print(schIDtoremove_list)

[576803, 577546, 5700110953]


In [66]:
new_school_details_df = school_details_df[(school_details_df.Zip_Codes != 63045) & (school_details_df.Zip_Codes != 63140) & (school_details_df.Zip_Codes != 63155)]
new_school_details_df

,educationClimateIndex,advancedPlacement,beforeandafterschoolprgms,blueribbonschool,charterschools,giftedandtalented,internationbaccalaureate,magnetschool,sitebased,collegebound,...,adultother,specialeducation,alterantiveprogram,yearroundclasses,ESL,Povertylevel,Instructionalexpensepupil,AYPschool,OBInstID,Zip_Codes
0,BELOW AVERAGE,Y,N,N,N,N,N,N,N,91,...,None,Y,N,N,Y,6 - 15.9 PERCENT,5669,N,577405,63123
1,ABOVE AVERAGE,Y,Y,N,N,Y,N,N,N,98,...,None,Y,N,N,N,6 - 15.9 PERCENT,6840,N,579439,63011
2,ABOVE AVERAGE,Y,N,N,N,Y,N,N,Y,95,...,Y,Y,N,N,Y,6 - 15.9 PERCENT,6840,Y,1524889,63021
3,BELOW AVERAGE,N,N,N,N,N,N,N,N,77,...,None,Y,N,N,N,6 - 15.9 PERCENT,4795,N,579881,63137
4,LOW,Y,N,N,N,N,N,N,N,52,...,None,Y,N,N,Y,30 PERCENT OR MORE,7305,N,582008,63118
5,BELOW AVERAGE,Y,N,N,N,Y,N,N,N,85,...,None,Y,Y,N,Y,6 - 15.9 PERCENT,5227,N,579738,63134
6,ABOVE AVERAGE,Y,N,N,N,Y,N,N,N,96,...,None,Y,N,N,Y,6 - 15.9 PERCENT,9590,Y,577625,63144
7,AVERAGE,N,Y,N,N,N,N,N,N,87,...,Y,Y,N,N,Y,6 - 15.9 PERCENT,7736,N,579544,63044
8,ABOVE AVERAGE,N,Y,N,N,N,N,N,Y,0,...,None,Y,N,Y,N,6 - 15.9 PERCENT,0,N,5700078207,63005
9,ABOVE AVERAGE,Y,N,N,N,Y,N,N,N,96,...,None,Y,N,N,Y,6 - 15.9 PERCENT,5763,N,4032500,63017


In [67]:
# Write the school details data to a csv
new_school_details_df.to_csv("..\Data\school_detail3.csv", encoding="utf-8", index=False)